In [5]:
import sys
import pathlib
pth=str(pathlib.Path().absolute())
print(pth)
sys.path.append(('\\').join(pth.split('\\')[:-1])+"\\Utils")
from Utilities import *
from MF import *
from Visualization_utilities import *
from transformation import Transformation
from Signatures import Signature
import os

e:\INNOVATE\proyectox\Simetrias\Notebooks


In [2]:
def project_pcl_pca(pcl,voxel_size=0.08,nb=5,std_ratio=1.5,p_th=0.5,ig="sphericity",upper=True):
    #o3d.visualization.draw_geometries([pcl])
    pcl.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamKNN(30))
    pcl, ind = pcl.remove_statistical_outlier(nb_neighbors=nb,std_ratio=std_ratio)
    pcl=pcl.voxel_down_sample(voxel_size=voxel_size)
    pcl.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamKNN(30))
    #o3d.visualization.draw_geometries([pcl])

    indexes=np.arange(np.asarray(pcl.points).shape[0])
    pt = o3d.geometry.KDTreeFlann(pcl)
    signatures=Signature(pcl,pt,indexes)
    Signature().v_build(signatures,NN_Criteria="KNN",
                    rad=np.std(np.asarray(pcl.points)-np.mean(np.asarray(pcl.points),axis=0))/8,
                   NN=30)

    signaturesp=remove_discontinuities_by_curvature(pcl,signatures,pt,curvature=0.4,radius=0.5)
    #graficarPropiedad(signaturesp,pcl,property=ig,frac=1)

    #LESS THAN
    pp=np.vectorize(lambda signature,att: getattr(signature,att))(signaturesp,ig)
    if upper:
        pp=np.delete(signaturesp,pp>p_th)
    else:
        pp=np.delete(signaturesp,pp<p_th)
    points=np.vectorize(lambda x: x.pointCoordinates,signature="()->(j)")(pp)
    
    data_mean = np.mean(points, axis=0) #Calculate the average value of the column
    # Normalized 
    normalize_data = points - data_mean
    # SVD decomposition
    # Construct covariance matrix
    H = np.dot(normalize_data.T, normalize_data)
    # SVD decomposition
    eigenvectors, eigenvalues, eigenvectors_t = np.linalg.svd(H)   # H = U S V
    # Reverse order
    sort = eigenvalues.argsort()[::-1]
    eigenvalues = eigenvalues[sort]
    eigenvectors = eigenvectors[:, sort]
    return eigenvectors[:, 0],eigenvectors[:, 1]

In [3]:
pts_lts=np.array(os.listdir("E:\\INNOVATE\\Aceptado"))

In [4]:
#Afinación de parametros
#pts = "A1_MODELO_3D_revA_L3x3x3_1600_pulg_001.pts"
pts = "A1_MODELO_3D_revA_C3X600_pulg_005.pts"  #Raro
#pts='A1_MODELO_3D_revA_L5x3x1_400_pulg_001.pts'
dpf=o3d.io.read_point_cloud("E:\\INNOVATE\\Aceptado\\"+pts)


pts,obb=project_pcl_(dpf,
             voxel_size=0.02,
             nb=5,
             std_ratio=1.5,
             p_th=0.1,
             ig="Linearity",
             upper=True
            )
mesh_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=80.0, origin=pts.get_center())
o3d.visualization.draw_geometries([pts, mesh_frame,obb])

pts,obb=project_pcl_(dpf,
             voxel_size=0.02,
             nb=5,
             std_ratio=1.5,
             p_th=0.65,
             ig="Planarity",
             upper=False
            )
mesh_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=80.0, origin=pts.get_center())
o3d.visualization.draw_geometries([pts, mesh_frame,obb])

pts,obb=project_pcl_(dpf,
             voxel_size=0.02,
             nb=5,
             std_ratio=1.5,
             p_th=0.75,
             ig="ratio",
             upper=False
            )
mesh_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=80.0, origin=pts.get_center())
o3d.visualization.draw_geometries([pts, mesh_frame,obb])

pts,obb=project_pcl_(dpf,
              voxel_size=0.02,
              nb=5,
              std_ratio=1.5,
              p_th=0.75,
              ig="Omnivariance",
              upper=False
             )
mesh_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=80.0, origin=pts.get_center())
o3d.visualization.draw_geometries([pts, mesh_frame,obb])

pts,obb=project_pcl_(dpf,
             voxel_size=0.02,
             nb=5,
             std_ratio=1.5,
             p_th=0.75,
             ig="Anistropy",
             upper=False
            )
mesh_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=80.0, origin=pts.get_center())
o3d.visualization.draw_geometries([pts, mesh_frame,obb])

TypeError: project_pcl_() got an unexpected keyword argument 'voxel_size'

In [ ]:
while True:
    dpf=np.random.choice(pts_lts)
    dpf=o3d.io.read_point_cloud("E:\\INNOVATE\\Aceptado\\"+dpf)
    pts=project_pcl_(dpf,
             voxel_size=0.02,
             nb=5,
             std_ratio=1.5,
             p_th=0.1,
             ig="Linearity",
             upper=True
            )
    mesh_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=80.0, origin=pts.get_center())
    o3d.visualization.draw_geometries([pts, mesh_frame])

    pts=project_pcl_(dpf,
             voxel_size=0.02,
             nb=5,
             std_ratio=1.5,
             p_th=0.65,
             ig="Planarity",
             upper=False
            )
    mesh_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=80.0, origin=pts.get_center())
    o3d.visualization.draw_geometries([pts, mesh_frame])

    pts=project_pcl_(dpf,
             voxel_size=0.02,
             nb=5,
             std_ratio=1.5,
             p_th=0.75,
             ig="ratio",
             upper=False
            )
    mesh_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=80.0, origin=pts.get_center())
    o3d.visualization.draw_geometries([pts, mesh_frame])

    pts=project_pcl_(dpf,
              voxel_size=0.02,
              nb=5,
              std_ratio=1.5,
              p_th=0.75,
              ig="Omnivariance",
              upper=False
             )
    mesh_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=80.0, origin=pts.get_center())
    o3d.visualization.draw_geometries([pts, mesh_frame])

    pts=project_pcl_(dpf,
             voxel_size=0.02,
             nb=5,
             std_ratio=1.5,
             p_th=0.75,
             ig="Anistropy",
             upper=False
            )
    mesh_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=80.0, origin=pts.get_center())
    o3d.visualization.draw_geometries([pts, mesh_frame])

0.5999749571677867
0.001008215799418615
0.9860881975614826
0.0012486634575777384
0.9989917842005814
0.4000250428322132
686.8185710820337
25.408128021018783
0.999109386031882
0.09458227675762662
0.5999984468752473
0.0022327591578204173
0.9953512892567546
0.002138285364496582
0.9977672408421796
0.4000015531247527
1397.5262417301615
4.697327314642835
0.999778049553111
0.12854865128528162
0.5999940050071402
0.001961527808909494
0.9971760658633926
0.0037102267061043456
0.9980384721910905
0.4000059949928599
276.9312335690031
13.962043813783932
0.9996571611540174
0.17076532478480252
0.5999059780134939
0.004784812632954124
0.9772908855028948
0.019294700099783686
0.9952151873670458
0.400747889747908
185.12861648020512
17.801567502906757
0.9997045077583389
0.12805705817421842
0.5999709526041469
0.0012554173953555666
0.9931421412937746
0.007773765692835039
0.9987445826046444
0.40002904739585304
722.1418701501929
14.255828968316608
0.9997467572240187
0.08850139368694274
0.5989581178657448
0.032901

In [301]:
import open3d as o3d
from circlehough.hough import main
#pts = "A1_MODELO_3D_revA_C3X600_pulg_005.pts"  #Raro
#pts='A1_MODELO_3D_revA_L5x3x1_400_pulg_001.pts'
#pts = 'Pipe_0p75_pulg_ac_14.pts'
#pts = 'Pipe_1_pulg_mb_306.pts'
#pts ='Pipe_4_pulg_ac_56.pts'
pts = 'Pipe_3_pulg_ac_132.pts'
#pts = 'Pipe_5_pulg_ac_230.pts'
#pts = 'Pipe_6_pulg_ac_71.pts'
#pts = 'Pipe_6_pulg_ac_292.pts'
dpf=o3d.io.read_point_cloud("E:\\INNOVATE\\DB_TRAINING_1024_PTS\\Pipe\\"+pts)


ev1,ev2=project_pcl_pca(dpf,
             voxel_size=0.02,
             nb=5,
             std_ratio=1.5,
             p_th=0.1,
             ig="Linearity",
             upper=True
            )

In [302]:
ev1, ev2

(array([-0.00957793, -0.01194843, -0.99988274]),
 array([ 0.24543132, -0.96937001,  0.00923281]))

In [303]:
np.linalg.norm(ev1, 2)

1.0

In [304]:
dpf

PointCloud with 1420 points.

In [305]:
o3d.visualization.draw_geometries([dpf])

In [306]:
def return_rot(vect, ang):
    cosA = np.cos(ang)
    sinA = np.sin(ang)
    oneMinusCosA = 1-cosA
    out = np.zeros((3, 3))          
    out[0, 0] = (cosA + vect[0] * vect[0] * oneMinusCosA)
    out[0, 1] = (vect[1] * vect[0] * oneMinusCosA + vect[2] * sinA)
    out[0, 2] = (vect[2] * vect[0] * oneMinusCosA - vect[1] * sinA)
    out[1, 0] = (vect[0] * vect[1] * oneMinusCosA - vect[2] * sinA)
    out[1, 1] = (cosA + vect[1] * vect[1] * oneMinusCosA)
    out[1, 2] = (vect[2] * vect[1] * oneMinusCosA + vect[0] * sinA)
    out[2, 0] = (vect[0] * vect[2] * oneMinusCosA + vect[1] * sinA)
    out[2, 1] = (vect[1] * vect[2] * oneMinusCosA - vect[0] * sinA)
    out[2, 2] = (cosA + vect[2] * vect[2] * oneMinusCosA)
    return out

In [307]:
aux1 = np.cross(ev1, np.array([0.0, 0.0, 1.0]))
aux1 = aux1/np.linalg.norm(aux1)
aux2 = ev1[2]/np.linalg.norm(ev1)
aux2 = np.arccos(aux2)
rot = return_rot(aux1, aux2)

In [308]:
cloud = dpf
mesh_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=80.0, origin=cloud.get_center())
cloud.rotate(rot)
o3d.visualization.draw_geometries([cloud, mesh_frame])

In [309]:
p = np.asarray(cloud.points)
INICIAL_CENTER = cloud.get_center()[:2]
INITIAL_RADIUS = (np.max(n_p[:, 1]) - np.min(n_p[:, 1]))*0.4
n_p = np.copy(p)


low_bound = np.min(n_p[:, 2])
high_bound = np.max(n_p[:, 2])
l = high_bound - low_bound
N_SLICES = 10
nd = {i:0 for i in range(N_SLICES)}
pd = {i:[] for i in range(N_SLICES)}
for cont, i in enumerate(n_p):
    for j in range(N_SLICES):
        if i[2]>= low_bound+j*l/N_SLICES and i[2] < low_bound+(j+1)*l/N_SLICES:
            nd[j] += 1
            pd[j].append([n_p[cont, 0], n_p[cont, 1]])
            n_p[cont, 2] = low_bound+j*l/N_SLICES
print(nd)
test = o3d.geometry.PointCloud()
test.points = o3d.utility.Vector3dVector(n_p)
o3d.visualization.draw_geometries([test])

{0: 82, 1: 159, 2: 188, 3: 174, 4: 177, 5: 153, 6: 165, 7: 138, 8: 150, 9: 33}


In [310]:
INICIAL_CENTER, INITIAL_RADIUS


(array([10842.10028169, 24890.67042254]), 69.52027235219603)

In [311]:
mm = []
for i in range(N_SLICES):
    pointss = np.array(pd[i])

    guessed_cx = INICIAL_CENTER[0]    
    guessed_cy = INICIAL_CENTER[1]
    guessed_r = INITIAL_RADIUS

# uncertainty of the initial guess
    uncertainty = 40.0

# width where points can still be counted to be part of the ring
    epsilon = 4.0

# perform the transformation
    hough_cx, hough_cy, hough_r = main(
        guessed_cx, guessed_cy, guessed_r, pointss,
        uncertainty, epsilon
    )
    mm.append(hough_r*2/25.6)
    #return found ring center and its radius
    print(hough_cx, hough_cy, hough_r*2/25.6, sep="===")
print(np.mean(np.array(mm)))

10845.101===24869.17===4.103146195411682
10848.601===24881.92===3.214474320411682
10836.132===24887.428===4.185543656349182
10847.976===24890.232===3.492794632911682
10850.351===24893.17===3.478146195411682
10850.491===24894.615===3.534298539161682
10852.726===24897.764===3.546505570411682
10853.476===24898.92===3.517208695411682
10855.241===24900.568===3.505001664161682
10846.601===24908.795===4.415646195411682
3.699276566505432


In [162]:
n_p

array([[-20643.31,    909.47,      0.  ],
       [-20634.31,    930.47,      0.  ],
       [-20745.31,    819.47,      0.  ],
       ...,
       [-20692.31,    883.47,      0.  ],
       [-20630.31,    937.47,      0.  ],
       [-20740.31,    834.47,      0.  ]])

In [93]:
test1.rotate(rot)

PointCloud with 6000 points.

In [95]:
o3d.visualization.draw_geometries([test1, mesh_frame])

In [63]:
b = 7
a = b
b += 5
print(b, a, sep='\n')

12
7


In [55]:
o3d.visualization.draw_geometries([test1, test2, test3])

In [56]:
guessed_cx = test1.get_center()[0]
guessed_cy = test1.get_center()[2]
guessed_r = 80

# uncertainty of the initial guess
uncertainty = 0.2

# width where points can still be counted to be part of the ring
epsilon = 10.0

# perform the transformation
hough_cx, hough_cy, hough_r = main(
    guessed_cx, guessed_cy, guessed_r, out[:6000, :2],
    uncertainty, epsilon
)

#return found ring center and its radius
print(hough_cx, hough_cy, hough_r, sep="===")

-20655.346===1721.4961===79.80039
